In [4]:
import tensorflow as tf
import numpy as np
import os

In [5]:
# Create a directory structure for the dumps
def create_dump_dirs(base_dir):
    os.makedirs(base_dir, exist_ok=True)
    os.makedirs(f"{base_dir}/inputs", exist_ok=True)
    os.makedirs(f"{base_dir}/outputs", exist_ok=True)
    os.makedirs(f"{base_dir}/weights", exist_ok=True)

In [6]:
# Save data to a binary file
def save_binary(data, filepath):
    data = np.array(data)
    with open(filepath, 'wb') as f:
        data.tofile(f)

In [7]:
# Layer-wise dump function
def layerwise_dump(model, input_sample, dump_dir):
    create_dump_dirs(dump_dir)
    intermediate_model = tf.keras.models.Model(inputs=model.input, outputs=[layer.output for layer in model.layers])
    
    # Disable XLA compilation for this predict call
    tf.config.run_functions_eagerly(True) 
    intermediate_outputs = intermediate_model.predict(input_sample)
    tf.config.run_functions_eagerly(False)  # Reset to default behavior

    # Iterate over all layers
    for i, layer in enumerate(model.layers):
        layer_name = layer.name

        # Save inputs for the layer (output of the previous layer)
        if i == 0:
            input_data = input_sample[0]  # Input to the first layer
        else:
            input_data = intermediate_outputs[i - 1]
        save_binary(input_data, f"{dump_dir}/inputs/{layer_name}_input.bin")

        # Save outputs of the layer
        output_data = intermediate_outputs[i]
        save_binary(output_data, f"{dump_dir}/outputs/{layer_name}_output.bin")

        # Save weights and biases if the layer has them
        if hasattr(layer, 'weights') and layer.weights:
            weights = layer.get_weights()
            for j, weight in enumerate(weights):
                save_binary(weight, f"{dump_dir}/weights/{layer_name}_weight_{j}.bin")

    print(f"Layer-wise dump completed. Data saved to {dump_dir}")

In [10]:
import tensorflow as tf
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import TFSMLayer
from tensorflow.keras.models import Model

if __name__ == "__main__":

    # Load the model using TFSMLayer
    model_layer = TFSMLayer('/Users/pravinpb/pycode/MCW/Assignments/cifar10-cpp/TensorRT_Models/cifar10_model_4', call_endpoint='serving_default')

    # Define the input shape
    input_shape = (32, 32, 3)
    inputs = Input(shape=input_shape)

    # Wrap TFSMLayer in a functional Keras model
    outputs = model_layer(inputs)
    model = Model(inputs=inputs, outputs=outputs)

    # Generate dummy input data
    input_sample = np.random.rand(1, 32, 32, 3).astype(np.float32)  # Batch size of 1

    # Perform the layer-wise dump
    dump_dir = "/content/drive/MyDrive/layer_dumps"
    layerwise_dump(model, input_sample, dump_dir)

RuntimeError: Op type not registered 'TRTEngineOp' in binary running on Pravins-MacBook-Air.local. Make sure the Op and Kernel are registered in the binary running in this process. Note that if you are loading a saved graph which used ops from tf.contrib (e.g. `tf.contrib.resampler`), accessing should be done before importing the graph, as contrib ops are lazily registered when the module is first accessed.